Load the database Social-Network.

In [1]:
%load_ext sql
%sql sqlite:///Social-Network.db

'Connected: @Social-Network.db'

**Assignment II Description**

Students at your hometown high school have decided to organize their social network using databases. So far, they have
collected information about sixteen students in four grades, 9-12. Here's the schema:

Highschooler ( ID, name, grade )
English: There is a high school student with unique _ID_ and a given _first name_ in a certain _grade_.

Friend ( ID1, ID2 )
English: The student with _ID1_ is friends with the student with _ID2_. Friendship is mutual, so if (123, 456) is in the Friend table, so is (456, 123).

Likes ( ID1, ID2 )
English: The student with _ID1_ likes the student with _ID2_. Liking someone is not necessarily mutual, so if (123, 456) is in the Likes table, there is no guarantee that (456, 123) is also present.

Your queries will run over a small data set conforming to the schema. You can view the database in the pdf file.

For your convenience, here is a graph showing the various connections between the students in our database. 9th graders
are blue, 10th graders are green, 11th graders are yellow, and 12th graders are purple. Undirected black edges indicate
friendships, and directed red edges indicate that one student likes another student.

![title](image.png)

**Instructions**

Each problem asks you to write a query in SQL. You can run your code by clicking Run button on the top of the page.

**Important Notes**

1- Your queries are executed using SQLite, so you must conform to the SQL constructs supported by SQLite.

2- Unless a specific result ordering is asked for, you can return the result rows in any order.

3- You are to translate the English into a SQL query that computes the desired result over all possible databases. I will give you the correct output two weeks later as a reference. However, just to check the answer is not enough. This means for our small sample database, even if your answer is correct, it is possible that your query does not correctly reflect the problem at hand. (For example, if we ask for a complex condition that requires accessing all of the tables, but over our small data set in the end the condition is satisfied only by Star Wars, then the query "select title from Movie where title = 'Star Wars'" will get correct answer even though it doesn't reflect the actual question.) Circumventing the system in this fashion is not a good idea to help you learn SQL. On the other hand, an incorrect attempt at a general solution is unlikely to produce the right answer, so you shouldn't be led astray by just checking the answer.

**Assignment Questions**

This assignment includes three exercises **[Social-Network Query Exercises], [SQL Social-Network Query Exercises Extras]** and **[Social-Network Modification Exercises]**. Finish all questions listed below and test if your answers are correct with reference output. Then fill your answers in this template.

**What to hand in:**
1. Generate a HTML report from this notebook template with answers (File > Download as > HTML).
2. This "Assignment_2.ipynb" file with answers.

Social-Network Query Exercises, Q1:

Find the names of all students who are friends with someone named Gabriel.

In [2]:
%%sql
SELECT Highschooler.name
FROM Highschooler, Friend
WHERE Highschooler.ID = Friend.ID2 
    AND Friend.ID1 IN (
        SELECT Highschooler.ID 
        FROM Highschooler
        WHERE Highschooler.name = "Gabriel"
    )
;

 * sqlite:///Social-Network.db
Done.


name
Cassandra
Andrew
Jessica
Jordan
Alexis


Social-Network Query Exercises, Q2:

For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and
the name and grade of the student they like.

In [3]:
%%sql
WITH
gradeIDMap1 AS (SELECT Highschooler.ID AS ID, Highschooler.name AS name, Highschooler.grade AS grade
FROM Highschooler, Likes
WHERE Highschooler.ID = Likes.ID1),

gradeIDMap2 AS (SELECT Highschooler.ID AS ID, Highschooler.name AS name, Highschooler.grade AS grade
FROM Highschooler, Likes
WHERE Highschooler.ID = Likes.ID2)

SELECT gradeIDMap1.name, gradeIDMap1.grade, gradeIDMap2.name, gradeIDMap2.grade
FROM gradeIDMap1, gradeIDMap2, Likes
WHERE gradeIDMap1.ID = Likes.ID1
    AND gradeIDMap2.ID = Likes.ID2
    AND ABS(gradeIDMap2.grade - gradeIDMap1.grade) >= 2;

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1
John,12,Haley,10


Social-Network Query Exercises, Q3:

For every pair of students who both like each other, return the name and grade of both students. Include each pair only once,
with the two names in alphabetical order.

In [4]:
%%sql
WITH
likes1 AS (
SELECT Highschooler.ID AS ID, Highschooler.name AS name, Highschooler.grade AS grade, Likes.ID2 AS likes1
FROM Highschooler, Likes
WHERE Highschooler.ID = Likes.ID1
),
likes2 AS (
SELECT Highschooler.ID AS ID, Highschooler.name AS name, Highschooler.grade AS grade, Likes.ID1 AS likes1
FROM Highschooler, Likes
WHERE Highschooler.ID = Likes.ID2
)

SELECT likes1.name, likes1.grade, likes2.name AS name1, likes2.grade AS grade1
FROM likes1, likes2, Likes
WHERE likes1.ID = Likes.ID2
    AND likes2.ID = Likes.ID1
    AND likes1.name < likes2.name
GROUP BY likes1.name, name1
ORDER BY likes1.name, likes2.name;

 * sqlite:///Social-Network.db
Done.


name,grade,name1,grade1
Cassandra,9,Gabriel,9
Jessica,11,Kyle,12


Social-Network Query Exercises, Q4:

Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades.
Sort by grade, then by name within each grade.

In [5]:
%%sql
SELECT Highschooler.name, Highschooler.grade
FROM Highschooler, Likes
WHERE Highschooler.ID NOT IN (
    SELECT Highschooler.ID
    FROM Highschooler, Likes
    WHERE Highschooler.ID = Likes.ID1
        OR Highschooler.ID = Likes.ID2
)
GROUP BY name, grade
ORDER BY grade, name;

 * sqlite:///Social-Network.db
Done.


name,grade
Jordan,9
Tiffany,9
Logan,12


Social-Network Query Exercises, Q5:

For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not
appear as an ID1 in the Likes table), return A and B's names and grades.

In [6]:
%%sql
WITH
studentA AS (
SELECT Likes.ID1 AS ID, Highschooler.name AS name, Highschooler.grade AS grade
FROM Likes, Highschooler
WHERE Likes.ID1 = Highschooler.ID
),
studentB AS (
SELECT Likes.ID2 AS ID, Highschooler.name AS name, Highschooler.grade AS grade
FROM Likes, Highschooler
WHERE Likes.ID2 = Highschooler.ID
    AND Likes.ID2 NOT IN (
        SELECT Likes.ID1
        FROM Likes
    )
)

SELECT studentA.name, studentA.grade, studentB.name AS name1, studentB.grade AS grade1
FROM studentA, studentB, Likes
WHERE studentB.ID = Likes.ID2
    AND studentA.ID = Likes.ID1
GROUP BY studentA.name, studentA.grade, studentB.name, studentB.grade;

 * sqlite:///Social-Network.db
Done.


name,grade,name1,grade1
Alexis,11,Kris,10
Austin,11,Jordan,12
Brittany,10,Kris,10
John,12,Haley,10


Social-Network Query Exercises, Q6:

Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name
within each grade.

In [7]:
%%sql
SELECT Highschooler1.name, Highschooler1.grade
FROM Highschooler AS Highschooler1
WHERE Highschooler1.ID NOT IN (
    -- Find all the friends in a different grade
    SELECT Friend.ID1
    FROM Highschooler AS Highschooler2, Friend
    WHERE Friend.ID1 = Highschooler1.ID
        AND Friend.ID2 = Highschooler2.ID
        AND Highschooler1.grade != Highschooler2.grade
)
GROUP BY grade, name
ORDER BY grade, name;

 * sqlite:///Social-Network.db
Done.


name,grade
Jordan,9
Brittany,10
Haley,10
Kris,10
Gabriel,11
John,12
Logan,12


Social-Network Query Exercises, Q7:

For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can
introduce them!). For all such trios, return the name and grade of A, B, and C.

In [8]:
%%sql
WITH
HighschoolerCopy1 AS (
SELECT *
FROM Highschooler
),
HighschoolerCopy2 AS (
SELECT *
FROM Highschooler
),
HighschoolerCopy3 AS (
SELECT *
FROM Highschooler
),
FriendCopy1 AS (
SELECT *
FROM Friend
),
FriendCopy2 AS (
SELECT *
FROM Friend
)

SELECT DISTINCT HighschoolerCopy1.name, HighschoolerCopy1.grade,
HighschoolerCopy2.name AS name1, HighschoolerCopy2.grade AS grade1,
HighschoolerCopy3.name AS name2, HighschoolerCopy3.grade AS grade2
FROM HighschoolerCopy1, HighschoolerCopy2, HighschoolerCopy3,
Likes, FriendCopy1, FriendCopy2
WHERE HighschoolerCopy2.ID NOT IN (
    SELECT Friend.ID2
    FROM Friend
    WHERE Friend.ID1 = HighschoolerCopy1.ID
)
AND (HighschoolerCopy1.ID = Likes.ID1 AND HighschoolerCopy2.ID = Likes.ID2)
AND (HighschoolerCopy1.ID = FriendCopy1.ID1 AND HighschoolerCopy3.ID = FriendCopy1.ID2)
AND (HighschoolerCopy2.ID = FriendCopy2.ID1 AND HighschoolerCopy3.ID = FriendCopy2.ID2);

 * sqlite:///Social-Network.db
Done.


name,grade,name1,grade1,name2,grade2
Andrew,10,Cassandra,9,Gabriel,9
Austin,11,Jordan,12,Andrew,10
Austin,11,Jordan,12,Kyle,12


Social-Network Query Exercises, Q8:

Find the difference between the number of students in the school and the number of different first names.

In [9]:
%%sql
SELECT COUNT(Highschooler.ID) - COUNT(DISTINCT(Highschooler.name))
FROM Highschooler;

 * sqlite:///Social-Network.db
Done.


COUNT(Highschooler.ID) - COUNT(DISTINCT(Highschooler.name))
2


Social-Network Query Exercises, Q9:

Find the name and grade of all students who are liked by more than one other student.

In [10]:
%%sql
SELECT Highschooler.name, Highschooler.grade
FROM Highschooler
WHERE Highschooler.ID IN (
    SELECT Likes.ID2
    FROM Likes
    GROUP BY Likes.ID2
    HAVING count() >= 2
);

 * sqlite:///Social-Network.db
Done.


name,grade
Cassandra,9
Kris,10


Social-Network Query Exercises Extras, Q1:

For every situation where student A likes student B, but student B likes a different student C, return the names and grades of
A, B, and C.

In [11]:
%%sql
WITH
LikesCopy1 AS (
SELECT *
FROM Likes
),
LikesCopy2 AS (
SELECT *
FROM Likes
),
HighschoolerCopy1 AS (
SELECT *
FROM Highschooler
),
HighschoolerCopy2 AS (
SELECT *
FROM Highschooler
),
HighschoolerCopy3 AS (
SELECT *
FROM Highschooler
)

SELECT HighschoolerCopy1.name, HighschoolerCopy1.grade,
HighschoolerCopy2.name, HighschoolerCopy2. grade,
HighschoolerCopy3.name, HighschoolerCopy3.grade
FROM HighschoolerCopy1, HighschoolerCopy2, HighschoolerCopy3, LikesCopy1, LikesCopy2
WHERE 
HighschoolerCopy1.ID = LikesCopy1.ID1
AND HighschoolerCopy2.ID = LikesCopy1.ID2
AND HighschoolerCopy3.ID = LikesCopy2.ID2

AND HighschoolerCopy2.ID = LikesCopy2.ID1
AND HighschoolerCopy1.ID != HighschoolerCopy3.ID;

 * sqlite:///Social-Network.db
Done.


name,grade,name_1,grade_1,name_2,grade_2
Andrew,10,Cassandra,9,Gabriel,9
Gabriel,11,Alexis,11,Kris,10


Social-Network Query Exercises Extras, Q2:

Find those students for whom all of their friends are in different grades from themselves. Return the students' names and
grades.

In [12]:
%%sql
SELECT Highschooler1.name, Highschooler1.grade
FROM Highschooler AS Highschooler1
WHERE Highschooler1.ID NOT IN (
    SELECT Friend.ID1
    FROM Highschooler AS Highschooler2, Friend
    WHERE Friend.ID1 = Highschooler1.ID
        AND Friend.ID2 = Highschooler2.ID
        AND Highschooler1.grade = Highschooler2.grade
);

 * sqlite:///Social-Network.db
Done.


name,grade
Austin,11


Social-Network Query Exercises Extras, Q3:

What is the average number of friends per student? (Your result should be just one number.)

In [13]:
%%sql
WITH friendMap AS (
SELECT Friend.ID1, count() AS count
FROM Friend
GROUP BY Friend.ID1
)

SELECT AVG(friendMap.count)
FROM friendMap;

 * sqlite:///Social-Network.db
Done.


AVG(friendMap.count)
2.5


Social-Network Query Exercises Extras, Q4:

Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count
Cassandra, even though technically she is a friend of a friend.

In [14]:
%%sql
WITH cassandraFriends AS (
SELECT Friend.ID2 AS ID
FROM Friend, Highschooler
WHERE Friend.ID1 = Highschooler.ID
    AND Highschooler.name = 'Cassandra'
)

SELECT count(*)
FROM Friend, cassandraFriends
WHERE Friend.ID1 IN (cassandraFriends.ID);

 * sqlite:///Social-Network.db
Done.


count(*)
7


Social-Network Query Exercises Extras, Q5:

Find the name and grade of the student(s) with the greatest number of friends.

In [15]:
%%sql
WITH friendMap AS (
SELECT Friend.ID1, count() AS count
FROM Friend
GROUP BY Friend.ID1
)

SELECT Highschooler.name, Highschooler.grade
FROM friendMap, Highschooler
WHERE friendMap.ID1 = Highschooler.ID
AND friendMap.count = (
    SELECT MAX(friendMap.count)
    FROM friendMap
);

 * sqlite:///Social-Network.db
Done.


name,grade
Andrew,10
Alexis,11


Social-Network Modification Exercises, Q1:

It's time for the seniors to graduate. Remove all 12th graders from Highschooler.

In [16]:
%%sql
DELETE
FROM Highschooler
WHERE Highschooler.grade = 12;

 * sqlite:///Social-Network.db
4 rows affected.


[]

Social-Network Modification Exercises, Q2:

If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple.

In [17]:
%%sql
DELETE
FROM Likes
WHERE (Likes.ID1, Likes.ID2) IN (
    SELECT Likes.ID1, Likes.ID2
    FROM Likes
)
AND (Likes.ID2, Likes.ID1) NOT IN (
    SELECT Likes.ID1, Likes.ID2
    FROM Likes
)
AND (Likes.ID1, Likes.ID2) IN (
    SELECT Friend.ID1, Friend.ID2
    FROM Friend
);

 * sqlite:///Social-Network.db
2 rows affected.


[]

Social-Network Modification Exercises, Q3:

For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself. _(This one is a bit challenging; congratulations if you get it right.)_

In [18]:
%%sql
WITH
FriendCopy1 AS (
SELECT *
FROM Friend
),
FriendCopy2 AS (
SELECT *
FROM Friend
),
FriendCopy3 AS (
SELECT *
FROM Friend
)

INSERT
INTO Friend(ID1, ID2)
SELECT DISTINCT FriendCopy1.ID1, FriendCopy2.ID2
FROM FriendCopy1, FriendCopy2
WHERE FriendCopy1.ID2 = FriendCopy2.ID1
    AND FriendCopy1.ID1 != FriendCopy2.ID2
    AND (FriendCopy1.ID1, FriendCopy2.ID2) NOT IN FriendCopy3;

 * sqlite:///Social-Network.db
Done.


[]